# Attachment in pySBOL2

The purpose of the `Attachment` class is to serve as a general container for data files, especially experimental data files. It provides a means for linking files and metadata to SBOL designs.

`Attachment` objects have the following properties:
- `source` : The source property is REQUIRED and MUST contain a URI reference to the source file.
- `format` : The format property is OPTIONAL and MAY contain a URI that specifies the format of the attached file. It is RECOMMENDED that this URI refer to a term from the EMBRACE Data and Methods (EDAM) ontology.
- `size`: The size property is OPTIONAL and MAY contain a long indicating the file size in bytes.
- `hash` : The hash property is OPTIONAL and MAY contain a SHA-1 hash of the file contents represented as a hexadecimal digest.

In this tutorial, we will first create a dummy experimental data file, then attach it to an SBOL document using pySBOL2.

For more information on the `Attachment` class and its properties, check out page 53 of the SBOL 2.3.0 specifications which can be found at the following [link](https://sbolstandard.org/docs/SBOL2.3.0.pdf).

Create dummy data

In [2]:
import os
import hashlib
import csv

# filename
dummy_filename = "plate_reader_exp1.csv"

# dummy data: Time, OD600, and GFP Fluorescence
data = [
    ["Time (h)", "OD600", "GFP_Fluorescence (AU)"],
    [0, 0.05, 100],
    [1, 0.10, 150],
    [2, 0.20, 500],
    [3, 0.40, 2000],
    [4, 0.80, 5000],
    [5, 1.20, 8000],
    [6, 1.50, 9500],
    [7, 1.60, 10000],
    [8, 1.65, 10100]
]

# write dummy data
with open(dummy_filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)

# 'size' property
file_size = os.path.getsize(dummy_filename)

# 'hash' property
hasher = hashlib.sha1()
# Read the file in binary mode ('rb') for hashing
with open(dummy_filename, 'rb') as f:
    # Read and update hash string value in blocks
    for chunk in iter(lambda: f.read(4096), b""):
        hasher.update(chunk)
file_hash_sha1 = hasher.hexdigest()

import the module

In [3]:
import sbol2

In [8]:
doc = sbol2.Document()

# Set a namespace for the document
sbol2.setHomespace('https://github.com/SynBioDex/SBOL-Notebooks')

Creating `Attachment` object

In [9]:
attachment = sbol2.Attachment("exp1_growth_data")

# Required source property
attachment.source = "./plate_reader_exp1.csv"

# Optional properties using the metadata
attachment.format = "http://edamontology.org/format_3752" # EDAM for csv
attachment.size = file_size
attachment.hash = file_hash_sha1

# --- Add the Attachment to the Document ---
doc.addAttachment(attachment)

In [13]:
report = doc.validate()
if report == 'Valid.':
    doc.write('example_attachment.xml')
else :
    print(report)